In [1]:
using Revise
using Clang, Clang_jll

In [2]:
## for attaching lldb
getpid()

79985

In [4]:
## check custom libclang is loaded from LocalPreferences
Clang_jll.libclang

"/Users/nehal/src/exploration/llvm-project/llvm/cmake-build-debug/lib/libclang.dylib"

In [5]:
sdk_dir = `xcrun --show-sdk-path` |> open |> readchomp |> String

"/Applications/Xcode.app/Contents/Developer/Platforms/MacOSX.platform/Developer/SDKs/MacOSX.sdk"

In [6]:
index = Index()

Index(Ptr{Nothing} @0x000000016761c600, 0, 1)

In [7]:
## needs patched clang.jl and patched libclang (due to typo, deletes file immediately after operation)
Clang.setInvocationEmissionPath(index,pwd())

In [7]:
trans_unit = Clang.parse_header(index, "050_test.h", ["-x","objective-c","-fmodules", "-isysroot", sdk_dir])

TranslationUnit(Ptr{Clang.LibClang.CXTranslationUnitImpl} @0x000000015263b540, Index(Ptr{Nothing} @0x0000000116010e10, 0, 1))

In [8]:
root_cursor = Clang.getTranslationUnitCursor(trans_unit)

CLCursor (CLTranslationUnit) 050_test.h

In [ ]:
dumpobj(root_cursor)

In [10]:
Clang.parse_header(index, "050_testa.h") |>  Clang.getTranslationUnitCursor |> dumpobj

 CLTranslationUnit 050_testa.h
   CLFunctionDecl c()
   CLFunctionDecl b()
   CLStructDecl 
      CLFieldDecl a CLInt int a
      CLFieldDecl a CLInt int b
   CLTypedefDecl Bar
     CLStructDecl 
        CLFieldDecl a CLInt int a
        CLFieldDecl a CLInt int b


In [11]:
Clang.parse_header(index, "050_testa.h") |>  Clang.getTranslationUnitCursor |> children

4-element Vector{CLCursor}:
 CLCursor (CLFunctionDecl) c()
 CLCursor (CLFunctionDecl) b()
 CLCursor (CLStructDecl) 
 CLCursor (CLTypedefDecl) Bar

In [16]:
args = [
    "-x","objective-c",
    "-isysroot", sdk_dir,
    # "-fmodules", 
    # "-fmodules-cache-path=cache",
    # "-fimplicit-module-maps",
    # #"-fmodules-validate-system-headers", 
    "-fblocks", 
    "-fregister-global-dtors-with-atexit",
    "-fgnuc-version=4.2.1", 
    "-fobjc-runtime=macosx-13.0.0", 
    "-fobjc-exceptions", 
    "-fexceptions",
    "-fmax-type-align=16",
    "-fcommon", 
    "-resource-dir",
    "/Users/nehal/.julia/artifacts/c9828f0ef0f967487ce376a3d8b4c8a646d38ace/lib/clang/13.0.1",
    "-DNS_FORMAT_ARGUMENT(A)=",
    "-D__GCC_HAVE_DWARF2_CFI_ASM=1",
    "-v",
    ]

    """
    A bit of explanation on what is going on.
    In Xcode 13 RC localizedAttributedStringForKey:value:table: method got NS_FORMAT_ARGUMENT(1) macro attribute. Previously, this attribute was applicable only to functions that return a C string, CFString or NSString. Recently, Apple added support in Clang for another type, NSAttributedString. Clang that ships with Kotlin/Native does not have this patch and fails to process localizedAttributedStringForKey:value:table: declaration.
    What this workaround does is it makes NS_FORMAT_ARGUMENT(1) a no-op. It is perfectly fine in this case since cinterop does not take this attribute into account anyway.
    https://github.com/apple/llvm-project/commit/2de0a18a8949f0235fb3a08dcc55ff3aa7d969e7
    
    """

trans_unit = Clang.parse_header(index, "050_import.h", args)

clang version 13.0.1 (https://github.com/habemus-papadum/llvm-project.git 75e33f71c2dae584b13a7d1186ae0a038ba98838)
Target: arm64-apple-darwin22.1.0
Thread model: posix
InstalledDir: 
ignoring nonexistent directory "/Applications/Xcode.app/Contents/Developer/Platforms/MacOSX.platform/Developer/SDKs/MacOSX.sdk/usr/local/include"
ignoring nonexistent directory "/Applications/Xcode.app/Contents/Developer/Platforms/MacOSX.platform/Developer/SDKs/MacOSX.sdk/Library/Frameworks"
#include "..." search starts here:
#include <...> search starts here:
 /Users/nehal/.julia/artifacts/c9828f0ef0f967487ce376a3d8b4c8a646d38ace/lib/clang/13.0.1/include
 /Applications/Xcode.app/Contents/Developer/Platforms/MacOSX.platform/Developer/SDKs/MacOSX.sdk/usr/include
 /Applications/Xcode.app/Contents/Developer/Platforms/MacOSX.platform/Developer/SDKs/MacOSX.sdk/System/Library/Frameworks (framework directory)
End of search list.


TranslationUnit(Ptr{Clang.LibClang.CXTranslationUnitImpl} @0x0000000137772090, Index(Ptr{Nothing} @0x000000016761c600, 0, 1))

In [17]:
trans_unit |> Clang.getTranslationUnitCursor |> children

15539-element Vector{CLCursor}:
 CLCursor (CLObjCClassRef) Protocol
 CLCursor (CLTypedefDecl) __int8_t
 CLCursor (CLTypedefDecl) __uint8_t
 CLCursor (CLTypedefDecl) __int16_t
 CLCursor (CLTypedefDecl) __uint16_t
 CLCursor (CLTypedefDecl) __int32_t
 CLCursor (CLTypedefDecl) __uint32_t
 CLCursor (CLTypedefDecl) __int64_t
 CLCursor (CLTypedefDecl) __uint64_t
 CLCursor (CLTypedefDecl) __darwin_intptr_t
 ⋮
 CLCursor (CLObjCClassRef) NSXPCConnection
 CLCursor (CLObjCInterfaceDecl) NSUserScriptTask
 CLCursor (CLTypedefDecl) NSUserScriptTaskCompletionHandler
 CLCursor (CLObjCInterfaceDecl) NSUserUnixTask
 CLCursor (CLTypedefDecl) NSUserUnixTaskCompletionHandler
 CLCursor (CLObjCInterfaceDecl) NSUserAppleScriptTask
 CLCursor (CLTypedefDecl) NSUserAppleScriptTaskCompletionHandler
 CLCursor (CLObjCInterfaceDecl) NSUserAutomatorTask
 CLCursor (CLTypedefDecl) NSUserAutomatorTaskCompletionHandler

In [ ]:
run(`$(clang()) -x objective-c 050_import.h -emit-ast -o test.pch -fmodules -isysroot $sdk_dir`);
nothing
#clang -x objective-c 050_import.h -emit-ast -o test.pch -fmodules

## TODO
-------------
- Clang.jl
  - debug why @import works on executable but not libclang (need to debug executable)
  - add module and file to cursor dump
  - try to add back implicit-modules

In [17]:
pwd()

"/Users/nehal/src/ObjectiveC.jl/exploration"

In [11]:
Clang.version()

"clang version 13.0.1 (/depot/downloads/clones/llvm-project.git-5a9787eb535c2edc5dea030cc221c1d60f38c9f42344f410e425ea2139e233aa 8a2ae8c8064a0544814c6fac7dd0c4a9aa29a7e6)"

In [12]:
Base.libllvm_version

v"13.0.1"

Invocation Log
------------------
```
{
    "toolchain": "/Users/nehal/src/exploration/llvm-project/llvm/cmake-build-debug",
    "libclang.operation": "parse",
    "libclang.opts": 0,
    "args": [
        "clang",
        "-fno-spell-checking",
        "-x",
        "objective-c",
        "-fmodules",
        "-isysroot",
        "/Applications/Xcode.app/Contents/Developer/Platforms/MacOSX.platform/Developer/SDKs/MacOSX.sdk",
        "050_test.h",
        "-fallow-editor-placeholders"
    ]
}
```